# DX 704 Week 7 Project

This week's project will investigate issues in a quadcopter controller based using a linear quadratic regulator.
You will start with an existing model of the system and logs from a quadcopter based on it, investigate discrepancies, and ultimately train a new model of the system dynamics.

The full project description and a template notebook are available on GitHub: [Project 7 Materials](https://github.com/bu-cds-dx704/dx704-project-07).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Introduction

You've just joined a drone startup.
On your first day, you join your new team to watch a test flight for a new quadcopter prototype.
Watching the prototype fly, the team comments that it is not as smooth as usual and suspects that something is off in the controller.
They provide you a copy of the dynamics model and log data from the prototype to investigate.

The quadcopter control model is a slightly more sophisticated version of the 1D quadcopter problem previously considered.

The state vector $\mathbf{x}_t$ now includes an acceleration component, and the action vector now has a servo control for the throttle instead of a raw force component.
\begin{array}{rcl}
\mathbf{x}_t & = & \begin{bmatrix} x_t \\ v_t \\ a_t \end{bmatrix} \\
\mathbf{u_t} & = & \begin{bmatrix} u_t \end{bmatrix}
\end{array}

## Part 1: Reconstruct the Control Matrix

You are provided the dynamics model in the files `model-A.tsv`, `model-B.tsv`, `cost-Q.tsv` and `cost-R.tsv`.
Recompute the control matrix $\mathbf{K}$ to be used in the infinite horizon linear quadratic regulator problem.

Save $\mathbf{K}$ in a file "control-K-intended.tsv" with columns x, v and a.

Submit "control-K-intended.tsv" in Gradescope.

In [5]:
import numpy as np 
import pandas as pd

def read_clean(path):
    df = pd.read_csv(path, sep="\t", header=0)  # read even if header present
    df = df.drop(columns=["index","Index","Unnamed: 0"], errors="ignore")  # drop index cols
    return df.apply(pd.to_numeric, errors="coerce").dropna(axis=1, how="all").values.astype(float)

A = read_clean("model-A.tsv")
B = read_clean("model-B.tsv")
Q = read_clean("cost-Q.tsv")
R = read_clean("cost-R.tsv")

from scipy.linalg import solve_discrete_are
P = solve_discrete_are(A, B, Q, R)
K = np.linalg.inv(B.T @ P @ B + R) @ (B.T @ P @ A)

cols = ["x","v","a"] if A.shape[1]==3 else [f"s{i}" for i in range(A.shape[1])]
pd.DataFrame(K, columns=cols).to_csv("control-K-intended.tsv", sep="\t", index=False)
print("Saved control-K-intended.tsv")


Saved control-K-intended.tsv


## Part 2: Recompute the Actions for the Logged States

You get access to the log data for the prototype as the file "qc-log.tsv".
It conveniently saves all the state and actions made.
Recompute the actions based on your $\mathbf{K}$ matrix computed in part 1.

In [ ]:
# YOUR CHANGES HERE
import pandas as pd
import numpy as np

log = pd.read_csv("qc-log.tsv", sep="\t")     
K = pd.read_csv("control-K-intended.tsv", sep="\t", index_col=0).values[0]  

X = log[["x", "v", "a"]].values
u_check = -X @ K                            

out = pd.DataFrame({"index": log.index, "u_check": u_check})
out.to_csv("qc-check.tsv", sep="\t", index=False)

print("Saved qc-check.tsv with columns 'index' and 'u_check'")

...

Saved qc-check.tsv with columns 'index' and 'u_check'


Ellipsis

Save your computed actions as "qc-check.tsv" with columns "index" and "u_check".

In [ ]:
# YOUR CHANGES HERE

...

Submit "qc-check.tsv" in Gradescope.

## Part 3: Reverse Engineer the Actual Control Matrix

Now that you have found a systematic difference between your computed actions and the logged actions, estimate $
$, the control matrix that was actually used to choose actions in the prototype.

Hint: With a linear quadratic regulator, the optimal actions are always linear combinations of the state that are calculated using the control matrix.
You can use linear regression to reverse-engineer the coefficients in the control matrix.

In [ ]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

log = pd.read_csv("qc-log.tsv", sep="\t")     

X = log[["x", "v", "a"]].values
y = log["u"].values

reg = LinearRegression(fit_intercept=False)
reg.fit(X, y)

K_actual = -reg.coef_   

K_df = pd.DataFrame([K_actual], columns=["x", "v", "a"], index=["u"])
K_df.to_csv("control-K-actual.tsv", sep="\t")

print("Saved control-K-actual.tsv")
print(K_df)

...

Saved control-K-actual.tsv
          x        v         a
u  0.340438  1.30012  1.950117


Ellipsis

Save $\mathbf{K}_{\mathrm{actual}}$ in "control-K-actual.tsv" with the same format as "control-K-intended.tsv".

In [ ]:
# YOUR CHANGES HERE

...

Submit "actual-K.tsv" in Gradescope.

## Part 4: Recompute the System Dynamics from the Log Data

On further investigation, it turns out that the control matrix $\mathbf{K}$ in the prototype was modified to compensate for state prediction errors.
You would like to recompute the $\mathbf{A}$ and $\mathbf{B}$ matrices using the log data since they are used to predict the next states.
However, since the action vector $\mathbf{u}_t$ is linearly dependent on the state via $\mathbf{u}_t=-\mathbf{K} \mathbf{x}_t$, you need a new data set so you can separate the effects of the $\mathbf{A}$ and $\mathbf{B}$ matrices.
Your co-workers kindly provide a new file "qc-train.tsv" where noise was added to each action.
Estimate the true $\mathbf{A}$ and $\mathbf{B}$ matrices based on this file.

In [ ]:
# YOUR CHANGES HERE
import numpy as np
import pandas as pd

df = pd.read_csv("qc-train.tsv", sep="\t")



df["x_next"] = df["x"].shift(-1)
df["v_next"] = df["v"].shift(-1)
df["a_next"] = df["a"].shift(-1)
df = df.dropna().reset_index(drop=True)

X = df[["x", "v", "a"]].values          
U = df[["u"]].values                    
Y = df[["x_next", "v_next", "a_next"]].values  

Z = np.hstack([X, U])                   
Theta, *_ = np.linalg.lstsq(Z, Y, rcond=None)  

A_new = Theta[:3, :]                    
B_new = Theta[3:, :].T                  

A_df = pd.DataFrame(A_new, index=["x","v","a"], columns=["x","v","a"])
B_df = pd.DataFrame(B_new, index=["x","v","a"], columns=["u"])

A_df.to_csv("model-A-new.tsv", sep="\t")
B_df.to_csv("model-B-new.tsv", sep="\t")


...

A_new:
               x             v             a
x  1.000000e+00 -1.281955e-16 -9.313780e-17
v  1.100000e+00  9.000000e-01  5.551115e-16
a  2.886580e-15  9.500000e-01  1.100000e+00

B_new:
               u
x -1.110223e-16
v -1.000000e-02
a  9.000000e-01


Ellipsis

Save $\mathbf{A}$ and $\mathbf{B}$ to "model-A-new.tsv" and "model-B-new.tsv" respectively.

In [ ]:
# YOUR CHANGES HERE

...

Submit "model-A-new.tsv" and "model-B-new.tsv" in Gradescope.

## Part 5: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 6: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.